In [2]:
import pandas as pd
import numpy as np
import xlwings as xw


TC_File_name = 'CstApPwrMGT_Integration_Testcase_v3.6_1.xlsm'
# test_comp_name = 'RCtApVBatStaChk_10ms_test0.csv'

df = pd.read_excel(TC_File_name,sheet_name = 'TestDesign')
# workbook = openpyxl.load_workbook('CstAp_CANMGT_Integration_Testcase_v3.6.xlsm', data_only = True)  오래걸림
df111 = pd.read_excel(TC_File_name,sheet_name = 'TestDataConvert')

wb = xw.Book(TC_File_name)
sheet = wb.sheets['TestDesign']

############################################################################################################################

# Input, output 찾고 인덱스 지정 -> input ouput 데이터 분리시키기 위함
index_in = df.columns.get_loc('Inputs')
index_out = df.columns.get_loc('Expected Outputs')
index_act = df.columns.get_loc('Actual Outputs')

#input 개수, output 개수
numinput = df111.iloc[4,1]
numoutput = df111.iloc[4,4]

# TC, input, ouput, stepdata 모음
TCdata = df.iloc[:,0]
P= TCdata.dropna()
inputdata = df.iloc[:,index_in:index_out]
outputdata = df.iloc[:,index_out:index_act]
stepdata = df.iloc[:,6]


# CASEID 위치 찾기
target_column_index = 0
target_column = df.iloc[:, target_column_index]

num = target_column.dropna().index.tolist()


outputID = df.iloc[0,index_out:index_act]

#파일이름 읽기
CSVname = df111.iloc[2,1]

#숫자를 행번호로 변환 ex) 1->'A' / 3-'C'  / 29 -> 'AC'

def number_to_excel_column(n):
    result = ""
    while n > 0:
        # 엑셀 열 문자로 변환
        remainder = (n - 1) % 26
        result = chr(ord('A') + remainder) + result
        n = (n - 1) // 26

    return result

StartActual = number_to_excel_column(10+numinput+numoutput)+'3'  # 시작넘버 input개수+output개수+10 +'3' =  'DC3'


##############################################################
#Step 모음
w=[]

for i in range(1,len(num)):
    w.append(stepdata[num[i]:num[i+1]])
    if i ==(len(num)-2):
        w.append(stepdata[num[i+1]:len(TCdata)])
        break

##############################################################
#폴더 내 모든 테스트케이스 파일 읽기 
tcfile = []

for i in range(0,len(num)-1):       # ex)0부터 58까지  -> range(0,59) 총 케이스개수 =  len(num)-1
    tcfile.append(pd.read_csv(f'{CSVname}_test{i}.csv',header=2))   #ex) tcfile[0] = test0, tcfile[1] = test1

##############################################################
# input,expect, Actual 중에서 Actual만 추출하기
tcfile_modi = []

for i in range(0,len(num)-1):
    tcfile_modi.append(tcfile[i].iloc[:,numinput+numoutput+4:])       # Actual값 시작 번호 = numinput + numoutput + 4
    tcfile_modi[i].columns = tcfile_modi[i].columns.str.replace('.1', '')  # 열 이름에 .1이 붙혀져서 제거필요 ex) DtcEnCndChk_SysPwrSta.1 -> DtcEnCndChk_SysPwrSta


##############################################################
# 순서 정렬  (디자인파일의 순서에 맞게  Actual 순서 재배열) 
tcfile_sort = []

for i in range(0,len(num)-1):
    tcfile_sort.append(tcfile_modi[i][outputID])


##############################################################
# 모든 값을 a 변수에 넣고 Y 데이터프레임 생성
a = []
for i in range(0,len(num)-1):
    for j in w[i]:
        a.append(tcfile_sort[i].iloc[j-1])


Y = pd.DataFrame(a)
##############################################################

# 값 넣기
sheet.range(StartActual).value = Y.values.tolist()

c:\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
